<a href="https://colab.research.google.com/github/ProfAI/tutorials/blob/master/nn_with_numpy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Una Rete Neurale da Zero
Ti svelo un segreto: non puoi usare le Reti Neurali Artificiali se non sai come funzionano le Reti Neurali. Quando ci si approccia a Deep Learning e alle Reti Neurali per la prima volta si viene subito intimoriti dalla matematica che c'è dietro e si finisce a guardare le API di Tensorflow o a fare copia-incolla di pezzi di codice presi in giro per il web.
<br>
Lo scopo di questo articolo è demistificare le Reti Neurali e mostrare che in fondo, se si sa come interpretarla, la matematica che c'è dietro non è nulla di terrificante.

## Prerequisiti
In questo articolo do per scontato che tu sappia cosa è e a cosa serve il Machine Learning, se così non fosse parti pure [da qui](http://blog.profession.ai/cosa-e-machine-learning/) e poi magari dai uno sguardo [a questo](http://blog.profession.ai/deep-learning-svelato-ecco-come-funzionano-le-reti-neurali-artificiali/).

In [0]:
import numpy as np

## Partiamo dalle Metriche
Le metriche sono un'argomento generale di un qualsiasi modello Machine Learning e non limitate alle sole Reti Neurali, queste ci permettono di determinare la qualità del nostro modello comparando le predizioni da esso fornite con i risultati reali presenti all'interno del dataset. Una metrica comune per problemi di classificazione è l'**accuracy**, che indica semplicemente la percentuale di predizioni che il nostro modello ha azzeccato ed è così definita:

$$ \frac{1}{N}\sum_{i=1}^N(\hat{y_i}-y_i)^2 $$

dove $\hat{y}$ sono le predizioni del modello mentre $y$ sono i valori reali. Definiamo una funzione per calcolare l'accuracy.



In [0]:
def accuracy(y, y_pred):
  return np.sum(y==y_pred)/len(y)

Uno dei limiti di questa metrica è che non tiene conto della *probabilità* che una predizione sia corretta, quindi un'errore grossolano assume lo stesso peso di un'errore minore. Per questo motivo è sempre una buona idea affiancare l'accuracy ad un altra metrica che tiene conto di questa informazione, la **Binary Cross Entropy** anche conosciuta come **Log Loss**, che è definita in questo modo:
<br><br>
$$ -\frac{1}{N}\sum_{i=1}^N(y\cdot log(a) + (1-y)\cdot(1-a) $$
dove $a$ sono le probabilità di appartenenza alla classe positiva ritornate dal modello, mentre $y$ sono sempre i valori reali. Definiamo una funzione per calcolare la Log Loss:

### Log Loss
$$ -\frac{1}{N}\sum_{i=1}^N(y\cdot log(a) + (1-y)\cdot(1-a) $$

In [0]:
def log_loss(y_true, y_proba):
  return -np.sum(y_true*np.log(y_proba)+(1-y_true)*np.log(1-y_proba))/len(y_true)

### Predizione
Passiamo ora alla fase di predizione, in una rete neurale la predizione avviene a cascata, l'input della rete arriva allo strato di input, viene moltiplicato per i pesi dello strato e viene sommato il bias in questo modo:
<br><br>
$$ z_1 = W_1x+b_1 $$
<br><br>
ora l'output dello strato di input diventerà l'input dello strato nascosto, questo processo è conosciuto come **Forward Progation** (Propagazione in Avanti) e queste sono le sue equazioni:

$ z_1 = W_1x+b_1  \\
 a_1 = \phi(z_1)  \\
 z_2 = W_2a_1+b_2  \\
 a_2 = \phi(z_2)  \\
$

come vedi all'output di uno strato viene applicata una funzione $\phi$, questa funzione è chimata **funzione di attivazione** e può variare da strato a strato. Cosa è una funzione di attivazione ? Una funzione di attivazione è una funzione che ci permette di aggiungere la non linearità alla nostra rete, senza di essa una rete neurale, anche una molto profonda, porterebbe agli stessi risultati di una regressione logistica. Esistono diverse funzioni di attivazione, per gli strati nascosti la più utilizzata e la Rectified Linear Unit (ReLu) che è così definita:

$$ \text{relu}(z) = \begin{Bmatrix} 
0  \ \ \ se \ \  z < 0
\\ 
z  \ \ \ se \ \ z \geq 0
\end{Bmatrix}
$$

Mentre per gli strati di output la funzione di attivazione da utilizzare dipende dal tipo di problema che stiamo affrontando, per una classificazione binaria bisogna usare la sigmoide, che è così definita.

$$ \text{sigmoid(z)} = \frac{1}{1+e^{-z}} $$

Implementiamo ReLu e Sigmoide in Python

In [0]:
def relu(Z):
  return np.maximum(Z, 0)

def sigmoid(Z):
  return 1/(1+np.power(np.e,-Z))

Aggiungendo le corrette funzioni di attivazione, le equazioni della forward propagtion diventano le seguenti.

$z_1 = W_1x+b_1 \\
a_1 = \text{relu}(z_1) \\
z_2 = W_2a_1+b_2 \\
a_2 = \text{sigmoid}(z_2) \\
$

Usiamo le equazioni per implementare la forward propagation in Python.

In [0]:
def forward_propagation(X, W1, b1, W2, b2):
  
  global cache
  
  Z1 = np.dot(X,W1)+b1
  
  A1 = relu(Z1)
  Z2 = np.dot(A1,W2)+b2
  A2 = sigmoid(Z2)
  
  cache = (Z1, A1, Z2, A2)
  
  return A2.ravel()

L'ultimo strato ci ritornerà la probabilità che l'osservazione in input appartenga alla classe positiva, una osservazione con una probabilità maggiore del 50% va classificata come appartenenza alla classe positiva, mentre un'osservazione con probabilità minore del 50% va classificata come appartenente alla classe negativa. Per standard, qualora la probabilità fosse esttamente del 50% classifichiamola come positiva, anche se non è attendibile, in ogni caso insieme ad una classificazione dobbiamo sempre prendere in considerazione la probabilità della sua corretteza.
$$
 \hat{y} = \begin{Bmatrix}  
1  \ \ \ se \ \  a_2\geq 0.5
\\ 
0  \ \ \ se \ \  a_2<0.5
\end{Bmatrix}
$$

Utilizziamo queste informazioni per creare la funzione di predizione. 

In [0]:
def predict(X, W1, b1, W2, b2):
  proba = forward_propagation(X, W1, b1, W2, b2)
  y = proba.copy()
  y[y>=0.5]=1
  y[y<0.5]=0
  return y

La fase di predizione è completa, passiamo all'addestramento, dove dovremo permettere alla nostra rete di apprendere i sui coefficienti: pesi e bias.

### Addestramento

L'addestramento della maggior parte dei modelli di machine learning si basa sull'utilizzo di un algoritmo di ottimizzazione, il più utilizzato è il **Gradient Descent**.
<br><br>
#### Il Gradient Descent in matematichese

Il funzionamento di questo algoritmo è abbastanza semplice: al valore dei coefficienti viene iterativamente sottratto il valore della corrispondente derivata parziale della funzione di costo moltiplicato per una costante, chiamata **learning rate**, e questo per un numero definito di cicli, chiamati **epoche**. Okay, detto così potrebbe non sembrare facile affatto, specialmente se non ricordi cosa è una derivata, facciamo un piccolo ripasso di analisi matematica. 
<br><br>
#### Derivate e Gradienti: un breve ripasso
La derivata di una funzione è un'altra funzione derivata dalla stessa funzione (e qui il nome) che indica quanto velocemente tale funzione sta crescendo/decrescendo in un determinato punto.  Se in un dato punto la funzione sta crescendo in maniera molto rapida, la sua derivata sarà un valore positivo grande, al contrario se la funzione sta descrescendo in maniera molto rapida la sua derivata sarà un valore negativo molto grande. Se invece la funzione è costante, quindi mantiene lo stesso valore, allora la derivatà varrà 0. Se una funzione ha più variabili allora ha più derivate, dato che ogni variabile può contribuire alla variazione della funzione in maniera differente, in questo caso si parla di variabili parziali, che messe insieme formano il gradiente della funzione. Ora che abbiamo rispolverato le derivate, torniamo al Gradient Descent.
<br><br>
#### Il Gradient Descent in italiano
In parole povere il Gradient Descent funziona così: i valori 'ideali' dei coefficienti sono quelli che ci permettono di ottenere il valore minore per la funzione di costo, cioè quelli che la minimizzano, sommando iterativamente il valore delle rispettive derivate parziali della funzione di costo tendiamo a 'spingere' i coefficienti verso tale punto di minimo.

In [0]:
def fit(X, epochs=100, lr=0.01):
     
  for _ in range(epochs):
    Y = predict(X, W1, b1, W2, b2)
    dW1, db1, dW2, db2 = funzione_magica_che_calcola_le_derivate_parziali(X, W2, A1, A2, Y)
    W1+=lr*dW1
    b1+=lr*db1
    W2+=lr*dW2
    b2+=lr*db2
    
  return W1, b1, W2, b2

Learning Rate e numero di Epoche sono due dei tanti **iperparametri** di una rete neurale, cioè quei valori che tocca a noi definire manualmente. Per una rete neurale il numero di epoche andrebbe sempre impostato almeno a 100, mentre il Learning Rate va cercato in un range esponeziale che va da 0.0001 a 10.
<br>
Come facciamo a calcolare il gradiente, cioè le derivate parziali della funzione di costo rispetto ai vari coefficienti ? Come si dice da me "e qua casca lo scecco" ! Se si fosse trattato di una regressione logistica, differenziare la funzione di costo sarebbe stato un gioco da ragazzi, ma nel caso di una rete neurale è molto più complesso, infatti una rete neurale è composta da più funzioni annidate, cioè funzioni che contengono altre funzioni, se non ci credi pensa che le equazioni della forward propagation possono anche essere espresse come un'unica equazione incomprensibile, questa qui:
<br><br>
$$ a_2 = \text{sigmoid}(W_2\cdot \text{relu}(W_1x+b_1)+b_2) $$
<br>
Ora noi dobbiamo riuscire a sapere quanto ogni coefficiente di ogni strato contribuisce all'errore della rete e questo problema non è per nulla banale ! Infatti gli scenziati ci si sono arrovellati sopra per 50 anni, fino al 1984, quando si arrivò ad una soluzione, la **backpropagation** (propagazione all'indietro o retropropagazione).
#### L'algoritmo della Backpropagation
La **backpropagation** è il processo inverso della forward propagation, questa volta l'output della rete va a ritroso dall'ultimo strato fino al primo. In realtà ad andare a ritroso non è l'output ma **l'errore** e in questo modo riusciamo a risalire a quanto ogni peso di ogni strato ha contribuito all'errore. Ma come ? Ma perché ? L'algoritmo si basa su una proprietà delle derivate, chiamata **Chain Rule** (Regola della Catena), che ci dice che la derivata di una funzione composta e pari al prodotto della derivata più esterna, avente come argomento la funzione interna, per la derivata della funzione interna. Quindi se abbiamo una funzione f(x) tale che:
<br><br>
$$f(x) = f(g(x)) $$
che quindi è una funzione composta, possiamo calcolare la sua derivata come
<br><br>
$$\frac{df}{dx}=\frac{ df}{df}\frac{dg}{dx}$$

Nel nostro caso abbiamo più funzioni annidate, esattamente 4 (il numero di equazioni della forward propagation), possiamo procedere così:
* Calcoliamo le derivate parziali della funzione di costo rispetto alla sigmoide
* Calcoliamo le derivate parziali della sigmoide rispetto a z2
* Calcoliamo le derivate parziali di z2 rispetto alla ReLu
* Calcoliamo le derivate parziali della ReLu rispetto a Z1
* Calcoliamo le derivate parizali di Z1 risetto ai coefficienti
* Moltiplichiamo tutti insieme

In forma matematichese abbiamo questo:
<br><br>
$$\frac{d\text{J}}{dw}=\frac{d\text{J}}{d\sigma}\frac{d\sigma}{dz_2}\frac{dz_2}{d\text{ReLu}}\frac{d\text{ReLu}}{dz_1}\frac{dz_1}{dw}$$
<br>
$$\frac{d\text{J}}{db}=\frac{d\text{J}}{d\sigma}\frac{d\sigma}{dz_2}\frac{dz_2}{d\text{ReLu}}\frac{d\text{ReLu}}{dz_1}\frac{dz_1}{db}$$

<br>
E come un salmone abbiamo ripercorso all'indietro la nostra rete neurale  e abbiamo ottenuto le derivate parziali della funzione di costo rispetto ai vari coefficienti. Questa ovviamente è un'equazione simbolica, per ottenere le equazioni effettive dobbiamo calcolare le varie derivate, cosa che ti risparmio, le derivate sono queste:
$\frac{d\text{J}}{dZ_2} = A_2-\hat{Y} \\
\frac{d\text{J}}{dW_2} = \frac{1}{N} (A{_2}^T \cdot \frac{d\text{J}}{dZ_2}) \\
\frac{d\text{J}}{db_2} = \frac{1}{N} \sum_{i=1}^N(\frac{d\text{J}}{dZ_2}) \\
\frac{d\text{J}}{dZ_1} = \frac{d\text{J}}{dZ_2} \cdot W_2^T * g'(Z_1) \\
\frac{d\text{J}}{dW_1} = \frac{1}{N} (X^T \cdot \frac{d\text{J}}{dZ_1})  \\
\frac{d\text{J}}{db_1} = \frac{1}{N} \sum_{i=1}^N(\frac{d\text{J}}{dZ_1}) \\
$
<br><br>
$g'(Z_1)$ è la derivata della funzione ReLu rispetto a $Z_1$, che è la seguente:
<br><br>
$$ g'(Z_1) =  \begin{Bmatrix} 
0  \ \ \ se \ \  z \leq 0
\\ 
1  \ \ \ se \ \ z > 0
\end{Bmatrix}
$$
<br>
Ora con queste equazioni davanti implemtiamo la backpropagation.

In [0]:
def relu_derivative(Z):
  Z[Z<=0] = 0
  Z[Z>0] = 1
  return Z

def back_propagation(X, W2, A1, A2, Y):
  
  m = A1.shape[1]
  dZ2 = A2.reshape(-1,1)-Y.reshape(-1,1)
  dW2 = np.dot(A1.T, dZ2)/m
  db2 = np.sum(dZ2, axis=0)
  
  m = X.shape[1]
  dZ1 = np.dot(dZ2.reshape(-1,1), W2.T)*relu_derivative(Z1)
  dW1 = np.dot(X.T, dZ1)/m
  db1 = np.sum(dZ2, axis=0)
  
  return dW1, db1, dW2, db2

All'interno della funzione fit sostituiamo la funzione_magica_per_calcolare_le_derivate_parziali con la back_propagation che abbiamo appena definito.

In [0]:
def fit(X, hidden, epochs=100, lr=0.01):
     
  for _ in range(epochs):
    Y = predict(X, W1, b1, W2, b2)
    dW1, db1, dW2, db2 = back_propagation(X, W2, A1, A2, Y)
    W1+=lr*dW1
    b1+=lr*db1
    W2+=lr*dW2
    b2+=lr*db2
    
  return W1, b1, W2, b2

Fantastico ! Manca un'ultima cosa, una funzione per inizializzare i valori dei coefficienti. Possiamo inizializzare i bias a 0, ma non i pesi ! Inizializzando i pesi a 0 le derivate parziali di tutti i pesi avranno lo stesso valore per tutte le iterazioni, questo vuol dire che il nostro modello non sarà migliore di un modello lineare. I pesi vanno inizializzati a valori casuali, nè troppo grandi nè troppo piccoli, infatti:
* Se i pesi vengono inizializzati a valori troppo grandi,  nel caso di una rete abbastanza profonda il gradiente diventerà ancora più grande, a causa delle varie moltiplicazioni tra valori elevati alla quale è soggetto, questo problema è chiamato **Exploding Gradient Problem** (Problema dell'esplosione del Gradiente).
* Se i pesi vengono inizializzati a valori troppo piccoli il problema è l'inverso, durante la backpropagation calcoleremo il gradiente eseguendo delle moltiplicazioni per valori molto piccoli, quindi questo tenderà a ridursi verso lo zero,  questo problema è chiamato **Vanishing Gradient Problem** (Problema della Scomparsa del Gradiente).

Esistono tecniche sofisticate per l'inizializzazione intelligente dei pesi, ma nel nostro caso stiamo realizzando una rete neurale con un solo strato nascosto, quindi non dovremmo preoccuparci di questi problemi, selezioniamo i pesi da una distribuzione normale.

In [0]:
def init_weights(input_size, hidden_size):
    
  W1 = np.random.randn(input_size, hidden_size) * 0.01
  b1 = np.zeros(hidden_size)
  W2 = np.random.randn(hidden_size,1) * 0.01
  b2 = np.zeros(1)
  
  return (W1, b1, W2, b2)

Aggiungiamo l'inizializzazione dei coefficienti all'interno della funzione fit

In [0]:
def fit(X, hidden, epochs=100, lr=0.01):
  
  W1, b1, W2, b2 = init_weights(X.shape[1], hidden)
   
  for _ in range(epochs):
    Y = predict(X, W1, b1, W2, b2)
    dW1, db1, dW2, db2 = back_propagation(X, W2, A1, A2, Y)
    W1+=lr*dW1
    b1+=lr*db1
    W2+=lr*dW2
    b2+=lr*db2
    
  return W1, b1, W2, b2

Questo è tutto ! La nostra rete neurale fatta in casa è pronta, testiamola sul campo.

## Testiamo la Rete Neurale
In [questo tutorial](http://blog.profession.ai/la-tua-prima-classificazione/), abbiamo creato un modello di regressione logistica per riconosere tumori al seno maligni, partendo da informazioni estratte da esami radiologici. Utilizziamo lo stesso dataset, questa volta faremo totalmente a meno di scikit-learn. Importiamo il dataset direttamente dalla [Repository Github dei tutorial di Profession.ai](https://github.com/ProfAI/tutorials), per farlo possiamo utilizzare Pandas, una popolare libreria Python per l'analisi dati.

In [111]:
import pandas as pd

breast_cancer = pd.read_csv("breast_cancer.csv", )
breast_cancer.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,radius error,texture error,perimeter error,area error,smoothness error,compactness error,concavity error,concave points error,symmetry error,fractal dimension error,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,malignant
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,1.0950,0.9053,8.589,153.40,0.006399,0.04904,0.05373,0.01587,0.03003,0.006193,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,0
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,0.5435,0.7339,3.398,74.08,0.005225,0.01308,0.01860,0.01340,0.01389,0.003532,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,0
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,0.7456,0.7869,4.585,94.03,0.006150,0.04006,0.03832,0.02058,0.02250,0.004571,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,0
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,0.4956,1.1560,3.445,27.23,0.009110,0.07458,0.05661,0.01867,0.05963,0.009208,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,0
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,0.7572,0.7813,5.438,94.44,0.011490,0.02461,0.05688,0.01885,0.01756,0.005115,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,0


Abbiamo in totale 31 colonne, cioè 30 features e un target, che è la colonna "malignant". Estraiamo features e target in array numpy.

In [0]:
X = breast_cancer.drop("malignant", axis=1).values
y = breast_cancer["malignant"].values

Ora dobbiamo creare gli array per addestramento e predizione, facciamo la nostra personalissima funzione train_test_split.

In [113]:
def train_test_split(X, y, test_size=0.3, random_state=None):

  if(random_state!=None):
    np.random.seed(random_state)
  
  n = X.shape[0]

  test_indices = np.random.choice(n, int(n*test_size), replace=False) # selezioniamo gli indici degli esempi per il test set
  
  # estraiamo gli esempi del test set
  # in base agli indici
  
  X_test = X[test_indices]
  y_test = y[test_indices]
  
  # creiamo il train set
  # rimuovendo gli esempi del test set
  # in base agli indici
  
  X_train = np.delete(X, test_indices, axis=0)
  y_train = np.delete(y, test_indices, axis=0)

  return (X_train, X_test, y_train, y_test )


X_train, X_test, y_train, y_test  = train_test_split(X, y, test_size=0.3)

print("Esempio in totale: %d" % X.shape[0])
print("Esempio per l'addestramento: %d" % X_train.shape[0])
print("Esempio per il test: %d" % X_test.shape[0])

Esempio in totale: 569
Esempio per l'addestramento: 399
Esempio per il test: 170


E' buona norma standardizzare o normalizzare i dati, per averli in una scala comune, questo può velocizzare anche di tanto la fase di addestramento. Optiamo per la normalizzazione (che è più semplice da implementare), la normalizzazione si esegue sottrando il valore minore e dividendo per la differenza tra il valore maggiore e il valore minore:
<br><br>
$$X_{norm} = \frac{X - X_min}{X_max-X_min}$$
<br><br>
Ricorda che dobbiamo **sempre applicare le stesse trasformazioni ai dati di addestramento, a quelli di test, e in generale a tutti quelli che daremo in pasto alla nostra rete neurale**,  quindi calcoliamo massimo e minimo sul set di addestramento e usiamo questi valori per la normalizzazione.

In [0]:
X_max = X_train.max(axis=0)
X_min = X_train.min(axis=0)

X_train = (X_train - X_min)/(X_max-X_min)
X_test = (X_test - X_min)/(X_max-X_min)

## La Rete Neurale in una Classe

In [0]:
class NeuralNetwork:
  
  
  def __init__(self, hidden_layer_size=100):
    
    self.hidden_layer_size=hidden_layer_size
    
    
  def _init_weights(self, input_size, hidden_size):
    
    self._W1 = np.random.randn(input_size, hidden_size)
    self._b1 = np.zeros(hidden_size)
    self._W2 = np.random.randn(hidden_size,1)
    self._b2 = np.zeros(1)

    
  def _accuracy(self, y, y_pred):      
    return np.sum(y==y_pred)/len(y)
  
  
  def _log_loss(self, y_true, y_proba):
    return -np.sum(np.multiply(y_true,np.log(y_proba))+np.multiply((1-y_true),np.log(1-y_proba)))/len(y_true)
  
  
  def _relu(self, Z):
    return np.maximum(Z, 0)

  
  def _sigmoid(self, Z):
    return 1/(1+np.power(np.e,-Z))
  
  
  def _relu_derivative(self, Z):
    Z[Z<=0] = 0
    Z[Z>0] = 1
    return Z
    
               
  def _forward_propagation(self, X):
                     
    Z1 = np.dot(X,self._W1)+self._b1

    A1 = self._relu(Z1)
    Z2 = np.dot(A1,self._W2)+self._b2
    A2 = self._sigmoid(Z2)
    
    self._forward_cache = (Z1, A1, Z2, A2)

    return A2.ravel()


  def predict(self, X, return_proba=False):

      proba = self._forward_propagation(X)

      y = np.zeros(X.shape[0])
      y[proba>=0.5]=1
      y[proba<0.5]=0

      if(return_proba):
        return (y, proba)
      else:
        return proba


  def predict_proba(self, X):         
      return self._forward_propagation(X)
                            
      
  def _back_propagation(self, X, y):
  
    Z1, A1, Z2, A2 = self._forward_cache
                   
    m = A1.shape[1]
    
    dZ2 = A2-y.reshape(-1,1)
    dW2 = np.dot(A1.T, dZ2)/m
    db2 = np.sum(dZ2, axis=0)/m

    dZ1 = np.dot(dZ2, self._W2.T)*self._relu_derivative(Z1)
    dW1 = np.dot(X.T, dZ1)/m
    db1 = np.sum(dZ1, axis=0)/m
    
    return dW1, db1, dW2, db2
           
               
  def fit(self, X, y, epochs=200, lr=0.01):
     
    self._init_weights(X.shape[1], self.hidden_layer_size)
      
    for _ in range(epochs):
      Y = self._forward_propagation(X)
      dW1, db1, dW2, db2 = self._back_propagation(X, y)
      self._W1-=lr*dW1
      self._b1-=lr*db1
      self._W2-=lr*dW2
      self._b2-=lr*db2
               

  def evaluate(self, X, y):
    y_pred, proba = self.predict(X, return_proba=True)
    accuracy = self._accuracy(y, y_pred)
    log_loss = self._log_loss(y, proba)
    return (accuracy, log_loss)

In [140]:
model = NeuralNetwork()
model.fit(X_train, y_train, epochs=500, lr=0.01)
model.evaluate(X_test, y_test)

(0.9529411764705882, 0.10412022044693808)